# KQML/KIF Agent Dialogue System
## Interactive Multi-Agent Communication Demonstration

**Agents:**
- Alice: Procurement agent (queries stock availability)
- Bob: Warehouse inventory agent (manages stock information)

**Communication Standards:**
- KQML (Knowledge Query and Manipulation Language)
- KIF (Knowledge Interchange Format)

---

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import seaborn as sns
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from enum import Enum
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")

## 1. KQML Performative Types

In [ ]:
class KQMLPerformative(Enum):
    ASK_IF = "ask-if"
    ASK_ONE = "ask-one"
    ASK_ALL = "ask-all"
    TELL = "tell"
    REPLY = "reply"
    SORRY = "sorry"
    ADVERTISE = "advertise"
    SUBSCRIBE = "subscribe"

## 2. KIF Knowledge Representation

In [ ]:
class KIFExpression:
    @staticmethod
    def predicate(name: str, *args) -> str:
        args_str = " ".join(str(arg) for arg in args)
        return f"({name} {args_str})"
    
    @staticmethod
    def and_expression(*predicates) -> str:
        preds_str = " ".join(predicates)
        return f"(and {preds_str})"
    
    @staticmethod
    def exists(variable: str, expression: str) -> str:
        return f"(exists ({variable}) {expression})"

## 3. KQML Message Structure

In [ ]:
@dataclass
class KQMLMessage:
    performative: KQMLPerformative
    sender: str
    receiver: str
    content: str
    reply_with: Optional[str] = None
    in_reply_to: Optional[str] = None
    language: str = "KIF"
    ontology: str = "warehouse-inventory"
    timestamp: str = None
    
    def __post_init__(self):
        if self.timestamp is None:
            self.timestamp = datetime.now().isoformat()
    
    def to_kqml_string(self) -> str:
        lines = [f"({self.performative.value}"]
        lines.append(f"  :sender {self.sender}")
        lines.append(f"  :receiver {self.receiver}")
        lines.append(f"  :content \"{self.content}\"")
        
        if self.reply_with:
            lines.append(f"  :reply-with {self.reply_with}")
        if self.in_reply_to:
            lines.append(f"  :in-reply-to {self.in_reply_to}")
        
        lines.append(f"  :language {self.language}")
        lines.append(f"  :ontology {self.ontology}")
        lines.append(")")
        
        return "\n".join(lines)
    
    def __str__(self) -> str:
        return self.to_kqml_string()

## 4. Product Database (Knowledge Base)

In [ ]:
class ProductDatabase:
    def __init__(self):
        self.inventory = {
            "tv-50inch-samsung": {
                "type": "television",
                "brand": "Samsung",
                "size": "50inch",
                "hdmi_slots": 4,
                "stock_level": 15,
                "available": True,
                "price": 599.99
            },
            "tv-50inch-lg": {
                "type": "television",
                "brand": "LG",
                "size": "50inch",
                "hdmi_slots": 3,
                "stock_level": 8,
                "available": True,
                "price": 549.99
            },
            "tv-50inch-sony": {
                "type": "television",
                "brand": "Sony",
                "size": "50inch",
                "hdmi_slots": 4,
                "stock_level": 0,
                "available": False,
                "price": 649.99
            }
        }
    
    def query_stock_level(self, product_id: str) -> Optional[int]:
        if product_id in self.inventory:
            return self.inventory[product_id]["stock_level"]
        return None
    
    def query_property(self, product_id: str, property_name: str) -> Optional[Any]:
        if product_id in self.inventory:
            return self.inventory[product_id].get(property_name)
        return None
    
    def query_products_by_criteria(self, **criteria) -> List[Dict]:
        matching_products = []
        
        for product_id, properties in self.inventory.items():
            match = True
            for key, value in criteria.items():
                if properties.get(key) != value:
                    match = False
                    break
            
            if match:
                product_info = {"product_id": product_id, **properties}
                matching_products.append(product_info)
        
        return matching_products

db = ProductDatabase()
df_inventory = pd.DataFrame.from_dict(db.inventory, orient='index')
print("\n📦 Current Inventory:")
display(df_inventory)

## 5. Agent Base Class

In [ ]:
class Agent:
    def __init__(self, name: str):
        self.name = name
        self.message_counter = 0
        self.conversation_history: List[KQMLMessage] = []
    
    def generate_message_id(self) -> str:
        self.message_counter += 1
        return f"{self.name.lower()}-msg-{self.message_counter:03d}"
    
    def send_message(self, message: KQMLMessage) -> None:
        self.conversation_history.append(message)
        print(f"\n{'='*70}")
        print(f"[{self.name} → {message.receiver}] @ {message.timestamp}")
        print('='*70)
        print(message.to_kqml_string())
        print()
    
    def receive_message(self, message: KQMLMessage) -> Optional[KQMLMessage]:
        self.conversation_history.append(message)
        return None

## 6. Alice Agent (Procurement Agent)

In [ ]:
class AliceAgent(Agent):
    def __init__(self):
        super().__init__("Alice")
        self.required_hdmi_slots = 4
        self.evaluated_products = []
    
    def ask_about_50inch_tv_stock(self, receiver: str) -> KQMLMessage:
        kif_content = KIFExpression.and_expression(
            KIFExpression.predicate("type", "?product", "television"),
            KIFExpression.predicate("size", "?product", "50inch"),
            KIFExpression.predicate("available", "?product", "true"),
            KIFExpression.predicate("stock-level", "?product", "?quantity")
        )
        
        message = KQMLMessage(
            performative=KQMLPerformative.ASK_ALL,
            sender=self.name,
            receiver=receiver,
            content=kif_content,
            reply_with=self.generate_message_id()
        )
        
        return message
    
    def ask_about_hdmi_slots(self, receiver: str, product_id: str, 
                            in_reply_to: str) -> KQMLMessage:
        kif_content = KIFExpression.predicate(
            "hdmi-slots", 
            product_id, 
            "?num_slots"
        )
        
        message = KQMLMessage(
            performative=KQMLPerformative.ASK_ONE,
            sender=self.name,
            receiver=receiver,
            content=kif_content,
            reply_with=self.generate_message_id(),
            in_reply_to=in_reply_to
        )
        
        return message
    
    def evaluate_product(self, product_info: Dict) -> bool:
        hdmi_slots = product_info.get("hdmi_slots", 0)
        stock_level = product_info.get("stock_level", 0)
        meets_requirements = hdmi_slots >= self.required_hdmi_slots and stock_level > 0
        
        self.evaluated_products.append({
            'product_id': product_info.get('product_id', 'Unknown'),
            'brand': product_info.get('brand', 'Unknown'),
            'hdmi_slots': hdmi_slots,
            'stock_level': stock_level,
            'price': product_info.get('price', 0),
            'meets_requirements': meets_requirements
        })
        
        return meets_requirements

## 7. Bob Agent (Warehouse Inventory Agent)

In [ ]:
class BobAgent(Agent):
    def __init__(self):
        super().__init__("Bob")
        self.database = ProductDatabase()
    
    def receive_message(self, message: KQMLMessage) -> Optional[KQMLMessage]:
        super().receive_message(message)
        
        if message.performative == KQMLPerformative.ASK_ALL:
            return self.handle_ask_all(message)
        elif message.performative == KQMLPerformative.ASK_ONE:
            return self.handle_ask_one(message)
        else:
            return KQMLMessage(
                performative=KQMLPerformative.SORRY,
                sender=self.name,
                receiver=message.sender,
                content="(unsupported-performative)",
                in_reply_to=message.reply_with
            )
    
    def handle_ask_all(self, message: KQMLMessage) -> KQMLMessage:
        products = self.database.query_products_by_criteria(
            type="television",
            size="50inch",
            available=True
        )
        
        if products:
            product_statements = []
            for product in products:
                product_id = product["product_id"]
                stock = product["stock_level"]
                brand = product["brand"]
                
                statement = KIFExpression.and_expression(
                    KIFExpression.predicate("product-id", product_id),
                    KIFExpression.predicate("brand", brand),
                    KIFExpression.predicate("stock-level", product_id, stock),
                    KIFExpression.predicate("available", product_id, "true")
                )
                product_statements.append(statement)
            
            kif_response = " ".join(product_statements)
        else:
            kif_response = "(no-matching-products)"
        
        response = KQMLMessage(
            performative=KQMLPerformative.REPLY,
            sender=self.name,
            receiver=message.sender,
            content=kif_response,
            in_reply_to=message.reply_with
        )
        
        return response
    
    def handle_ask_one(self, message: KQMLMessage) -> KQMLMessage:
        content = message.content
        
        if "hdmi-slots" in content:
            parts = content.strip("()").split()
            if len(parts) >= 2:
                product_id = parts[1]
                
                hdmi_slots = self.database.query_property(
                    product_id, 
                    "hdmi_slots"
                )
                
                if hdmi_slots is not None:
                    kif_response = KIFExpression.predicate(
                        "hdmi-slots",
                        product_id,
                        hdmi_slots
                    )
                else:
                    kif_response = "(unknown-product)"
            else:
                kif_response = "(invalid-query)"
        else:
            kif_response = "(unsupported-query)"
        
        response = KQMLMessage(
            performative=KQMLPerformative.REPLY,
            sender=self.name,
            receiver=message.sender,
            content=kif_response,
            in_reply_to=message.reply_with
        )
        
        return response

## 8. Visualization Functions

In [ ]:
def visualize_message_flow(messages: List[KQMLMessage]):
    fig, ax = plt.subplots(figsize=(14, 8))
    
    alice_x, bob_x = 2, 8
    y_spacing = 1.5
    start_y = len(messages) * y_spacing
    
    ax.add_patch(FancyBboxPatch((alice_x - 0.8, start_y + 0.5), 1.6, 1, 
                                boxstyle="round,pad=0.1", 
                                facecolor='lightblue', edgecolor='navy', linewidth=2))
    ax.text(alice_x, start_y + 1, 'ALICE\n(Procurement)', 
            ha='center', va='center', fontsize=11, fontweight='bold')
    
    ax.add_patch(FancyBboxPatch((bob_x - 0.8, start_y + 0.5), 1.6, 1, 
                                boxstyle="round,pad=0.1", 
                                facecolor='lightcoral', edgecolor='darkred', linewidth=2))
    ax.text(bob_x, start_y + 1, 'BOB\n(Warehouse)', 
            ha='center', va='center', fontsize=11, fontweight='bold')
    
    ax.plot([alice_x, alice_x], [0, start_y + 0.5], 'b--', linewidth=1.5, alpha=0.5)
    ax.plot([bob_x, bob_x], [0, start_y + 0.5], 'r--', linewidth=1.5, alpha=0.5)
    
    colors = {'ask-all': '#3498db', 'ask-one': '#2ecc71', 'reply': '#e74c3c', 'sorry': '#95a5a6'}
    
    for idx, msg in enumerate(messages):
        y_pos = start_y - (idx + 1) * y_spacing
        
        if msg.sender == "Alice":
            start_x, end_x = alice_x, bob_x
            direction = 'right'
        else:
            start_x, end_x = bob_x, alice_x
            direction = 'left'
        
        color = colors.get(msg.performative.value, '#34495e')
        
        arrow = FancyArrowPatch((start_x, y_pos), (end_x, y_pos),
                               arrowstyle='->', mutation_scale=20, 
                               linewidth=2.5, color=color, alpha=0.8)
        ax.add_patch(arrow)
        
        label = f"{msg.performative.value.upper()}"
        label_x = (start_x + end_x) / 2
        ax.text(label_x, y_pos + 0.3, label, 
               ha='center', va='bottom', fontsize=9, 
               bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=color, linewidth=1.5))
    
    ax.set_xlim(0, 10)
    ax.set_ylim(-1, start_y + 2)
    ax.axis('off')
    ax.set_title('Agent Communication Flow Diagram', fontsize=16, fontweight='bold', pad=20)
    
    legend_elements = [mpatches.Patch(facecolor=colors[k], label=k.upper()) 
                      for k in colors.keys()]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=10)
    
    plt.tight_layout()
    plt.show()

def visualize_inventory_status(database: ProductDatabase):
    df = pd.DataFrame.from_dict(database.inventory, orient='index')
    
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    brands = df['brand'].values
    stock = df['stock_level'].values
    colors_stock = ['green' if s > 0 else 'red' for s in stock]
    axes[0].barh(brands, stock, color=colors_stock, alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Stock Level', fontsize=11, fontweight='bold')
    axes[0].set_title('Stock Availability by Brand', fontsize=12, fontweight='bold')
    axes[0].grid(axis='x', alpha=0.3)
    
    hdmi = df['hdmi_slots'].values
    colors_hdmi = ['#2ecc71' if h >= 4 else '#e74c3c' for h in hdmi]
    axes[1].bar(brands, hdmi, color=colors_hdmi, alpha=0.7, edgecolor='black')
    axes[1].axhline(y=4, color='blue', linestyle='--', linewidth=2, label='Requirement (4 slots)')
    axes[1].set_ylabel('HDMI Slots', fontsize=11, fontweight='bold')
    axes[1].set_title('HDMI Slots Comparison', fontsize=12, fontweight='bold')
    axes[1].legend()
    axes[1].grid(axis='y', alpha=0.3)
    
    prices = df['price'].values
    axes[2].scatter(prices, stock, s=hdmi*100, c=colors_stock, alpha=0.6, edgecolors='black', linewidth=2)
    for i, brand in enumerate(brands):
        axes[2].annotate(brand, (prices[i], stock[i]), 
                        xytext=(5, 5), textcoords='offset points', fontsize=9)
    axes[2].set_xlabel('Price ($)', fontsize=11, fontweight='bold')
    axes[2].set_ylabel('Stock Level', fontsize=11, fontweight='bold')
    axes[2].set_title('Price vs Stock (size=HDMI slots)', fontsize=12, fontweight='bold')
    axes[2].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def visualize_evaluation_results(alice: AliceAgent):
    if not alice.evaluated_products:
        print("No products evaluated yet.")
        return
    
    df = pd.DataFrame(alice.evaluated_products)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    meets = df['meets_requirements'].value_counts()
    colors_pie = ['#2ecc71', '#e74c3c']
    labels_pie = ['Meets Requirements', 'Does Not Meet']
    axes[0, 0].pie(meets.values, labels=labels_pie, autopct='%1.1f%%', 
                   colors=colors_pie, startangle=90, textprops={'fontsize': 11})
    axes[0, 0].set_title('Product Evaluation Summary', fontsize=13, fontweight='bold')
    
    df_sorted = df.sort_values('meets_requirements', ascending=False)
    colors_bar = ['green' if m else 'red' for m in df_sorted['meets_requirements']]
    axes[0, 1].barh(df_sorted['brand'], df_sorted['hdmi_slots'], color=colors_bar, alpha=0.7, edgecolor='black')
    axes[0, 1].axvline(x=4, color='blue', linestyle='--', linewidth=2, label='Min Requirement')
    axes[0, 1].set_xlabel('HDMI Slots', fontsize=11, fontweight='bold')
    axes[0, 1].set_title('HDMI Slots vs Requirements', fontsize=13, fontweight='bold')
    axes[0, 1].legend()
    axes[0, 1].grid(axis='x', alpha=0.3)
    
    x = np.arange(len(df))
    width = 0.35
    axes[1, 0].bar(x - width/2, df['hdmi_slots'], width, label='HDMI Slots', alpha=0.8, color='skyblue', edgecolor='black')
    axes[1, 0].bar(x + width/2, df['stock_level']/2, width, label='Stock/2', alpha=0.8, color='lightcoral', edgecolor='black')
    axes[1, 0].set_xlabel('Products', fontsize=11, fontweight='bold')
    axes[1, 0].set_ylabel('Value', fontsize=11, fontweight='bold')
    axes[1, 0].set_title('HDMI Slots vs Stock Level', fontsize=13, fontweight='bold')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels(df['brand'], rotation=45)
    axes[1, 0].legend()
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    suitable = df[df['meets_requirements'] == True]
    if len(suitable) > 0:
        axes[1, 1].bar(suitable['brand'], suitable['price'], 
                      color='#3498db', alpha=0.7, edgecolor='black')
        axes[1, 1].set_ylabel('Price ($)', fontsize=11, fontweight='bold')
        axes[1, 1].set_title('Price Comparison (Suitable Products)', fontsize=13, fontweight='bold')
        axes[1, 1].grid(axis='y', alpha=0.3)
        
        for i, (brand, price) in enumerate(zip(suitable['brand'], suitable['price'])):
            axes[1, 1].text(i, price + 10, f'${price:.2f}', 
                          ha='center', va='bottom', fontweight='bold', fontsize=10)
    else:
        axes[1, 1].text(0.5, 0.5, 'No suitable products found', 
                       ha='center', va='center', transform=axes[1, 1].transAxes,
                       fontsize=12, color='red', fontweight='bold')
        axes[1, 1].set_title('Price Comparison (Suitable Products)', fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

def visualize_conversation_timeline(messages: List[KQMLMessage]):
    fig, ax = plt.subplots(figsize=(14, 6))
    
    performatives = [msg.performative.value for msg in messages]
    senders = [msg.sender for msg in messages]
    
    perf_counts = pd.Series(performatives).value_counts()
    
    x = np.arange(len(messages))
    colors_map = {'Alice': '#3498db', 'Bob': '#e74c3c'}
    colors = [colors_map.get(s, 'gray') for s in senders]
    
    ax.scatter(x, [1]*len(messages), s=300, c=colors, alpha=0.7, edgecolors='black', linewidth=2)
    
    for i, (msg, sender) in enumerate(zip(messages, senders)):
        ax.annotate(f"{msg.performative.value}\n({sender})", 
                   (i, 1), xytext=(0, -30 if i % 2 == 0 else 30), 
                   textcoords='offset points', ha='center',
                   fontsize=8, bbox=dict(boxstyle='round,pad=0.3', 
                   facecolor=colors[i], alpha=0.3, edgecolor='black'),
                   arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', lw=1.5))
    
    ax.set_ylim(0.5, 1.5)
    ax.set_xlim(-0.5, len(messages) - 0.5)
    ax.set_xlabel('Message Sequence', fontsize=12, fontweight='bold')
    ax.set_title('Conversation Timeline', fontsize=14, fontweight='bold')
    ax.set_yticks([])
    ax.grid(axis='x', alpha=0.3)
    
    legend_elements = [plt.Line2D([0], [0], marker='o', color='w', 
                                  markerfacecolor=colors_map[k], 
                                  markersize=10, label=k) for k in colors_map.keys()]
    ax.legend(handles=legend_elements, loc='upper right', fontsize=11)
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions loaded")

## 9. Dialogue Coordinator

In [ ]:
class DialogueCoordinator:
    def __init__(self):
        self.alice = AliceAgent()
        self.bob = BobAgent()
        self.all_messages: List[KQMLMessage] = []
    
    def deliver_message(self, sender: Agent, message: KQMLMessage, 
                       receiver: Agent) -> Optional[KQMLMessage]:
        sender.send_message(message)
        self.all_messages.append(message)
        
        response = receiver.receive_message(message)
        
        if response:
            receiver.send_message(response)
            self.all_messages.append(response)
        
        return response
    
    def run_dialogue(self) -> None:
        print("\n" + "="*70)
        print("AGENT DIALOGUE SIMULATION: ALICE AND BOB")
        print("Scenario: Stock Procurement Query")
        print("="*70)
        
        print("\n[STEP 1] Alice queries available 50-inch televisions...")
        msg1 = self.alice.ask_about_50inch_tv_stock("Bob")
        response1 = self.deliver_message(self.alice, msg1, self.bob)
        
        print("\n[STEP 2] Alice processes inventory information...")
        
        if response1 and response1.performative == KQMLPerformative.REPLY:
            available_products = self.bob.database.query_products_by_criteria(
                type="television",
                size="50inch",
                available=True
            )
            
            print(f"\nAlice found {len(available_products)} available product(s)")
            
            print("\n[STEP 3] Alice queries HDMI specifications...")
            
            for product in available_products:
                product_id = product["product_id"]
                
                msg_hdmi = self.alice.ask_about_hdmi_slots(
                    "Bob", 
                    product_id,
                    response1.reply_with or ""
                )
                response_hdmi = self.deliver_message(
                    self.alice, 
                    msg_hdmi, 
                    self.bob
                )
                
                if self.alice.evaluate_product(product):
                    print(f"\n✓ {product_id} meets requirements!")
                    print(f"  - Brand: {product['brand']}")
                    print(f"  - HDMI Slots: {product['hdmi_slots']}")
                    print(f"  - Stock Level: {product['stock_level']}")
                    print(f"  - Price: ${product['price']}")
                else:
                    print(f"\n✗ {product_id} does not meet requirements")
                    print(f"  - HDMI Slots: {product['hdmi_slots']} " +
                          f"(Required: {self.alice.required_hdmi_slots})")
        
        print("\n" + "="*70)
        print("DIALOGUE SUMMARY")
        print("="*70)
        print(f"Total messages exchanged: {len(self.all_messages)}")
        print(f"Alice sent: {len(self.alice.conversation_history)} messages")
        print(f"Bob sent: {len(self.bob.conversation_history)} messages")
        print("\nDialogue completed successfully!")
        print("="*70)

## 10. Initial Inventory Visualization

In [ ]:
temp_db = ProductDatabase()
print("\n📊 INITIAL WAREHOUSE INVENTORY STATUS")
print("="*70)
visualize_inventory_status(temp_db)

## 11. Run Agent Dialogue Simulation

In [ ]:
coordinator = DialogueCoordinator()
coordinator.run_dialogue()

## 12. Message Flow Visualization

In [ ]:
print("\n📈 AGENT COMMUNICATION FLOW DIAGRAM")
print("="*70)
visualize_message_flow(coordinator.all_messages)

## 13. Conversation Timeline

In [ ]:
print("\n⏱️ CONVERSATION TIMELINE ANALYSIS")
print("="*70)
visualize_conversation_timeline(coordinator.all_messages)

## 14. Product Evaluation Results

In [ ]:
print("\n🎯 ALICE'S PRODUCT EVALUATION RESULTS")
print("="*70)
visualize_evaluation_results(coordinator.alice)

## 15. Final Summary Statistics

In [ ]:
print("\n📊 DETAILED STATISTICS")
print("="*70)

msg_data = []
for msg in coordinator.all_messages:
    msg_data.append({
        'Sender': msg.sender,
        'Receiver': msg.receiver,
        'Performative': msg.performative.value,
        'Timestamp': msg.timestamp[:19]
    })

df_messages = pd.DataFrame(msg_data)
print("\n📨 Message Exchange Log:")
display(df_messages)

print("\n📋 Performative Distribution:")
perf_dist = df_messages['Performative'].value_counts()
display(perf_dist)

if coordinator.alice.evaluated_products:
    df_eval = pd.DataFrame(coordinator.alice.evaluated_products)
    print("\n🎯 Product Evaluation Summary:")
    display(df_eval)
    
    suitable = df_eval[df_eval['meets_requirements'] == True]
    print(f"\n✅ Products meeting requirements: {len(suitable)}/{len(df_eval)}")
    if len(suitable) > 0:
        best_product = suitable.loc[suitable['price'].idxmin()]
        print(f"\n💰 Best value product: {best_product['brand']}")
        print(f"   Price: ${best_product['price']:.2f}")
        print(f"   Stock: {best_product['stock_level']} units")
        print(f"   HDMI Slots: {best_product['hdmi_slots']}")

print("\n" + "="*70)
print("LEARNING OUTCOMES DEMONSTRATED")
print("="*70)
print("""
✓ Agent-Based Computing Motivations:
  - Autonomous decision making (Alice evaluates products)
  - Distributed knowledge (Bob maintains inventory database)
  - Communication and coordination between independent agents
  - Goal-oriented behavior (procurement task completion)

✓ Agent Communication Standards:
  - KQML for message structure and speech acts
  - KIF for knowledge representation
  - Asynchronous message passing
  - Conversation tracking with reply-with/in-reply-to

✓ Agent Models:
  - Reactive agents (Bob responds to queries)
  - Deliberative agents (Alice reasons about requirements)
  - Knowledge-based agents (both use structured databases)
  - Collaborative multi-agent systems

✓ Visual Analytics:
  - Message flow diagrams
  - Inventory status dashboards
  - Product evaluation matrices
  - Timeline analysis
""")
print("="*70)